# 史実を辿る「走れメロス」計算 ♯1 ('24.05.24)

太宰治「走れメロス」では、主人公メロスは、友人セリヌンティウスを人質として残し、シラクスの市街から十里(40km)離れた村まで3日間かけて往復する。
太宰が書いた「走れメロス」物語は、18世紀ドイツのフリードリヒ・フォン・シラーがしたためた書物の日本訳※をベースにしている。

太宰版「走れメロス」については、メロスの移動速度を検証した[優れた文献](https://www.rimse.or.jp/research/past/pdf/1st/work03.pdf)がある。
しかし、史実に遡った移動速度検証は試みられていない。
そこで、そうした計算をしてみようと思う。

※ 新編シラー詩抄 小栗孝則訳 改造文庫（昭和12年）

「走れメロス」は、そのルーツを辿ると、紀元前400年頃のイタリア、ディオニュシオス1世・2世の時代に遡る。
語られる内容にはバリエーションがあるが、比較的多いパターンは、ディオニュシオスから死刑を宣告されたピュティアスが友人ダモンを人質にして家まで往復をする、というものだ。
帰ってくるまでのタイムリミットは、当日の日暮れまでというものもあれば、三日後の日暮れとか数日後の日暮れまでというものもある。

%<img style="float:center;transform: rotate(0deg); height:10cm" src="./images/day_240524_geocoder_meros_Damon_and_Pythias.gif" />

```{figure} ./images/day_240524_geocoder_meros_Damon_and_Pythias.gif
---
height: 10cm
---
The story of Greeksの挿絵
```

## 竹馬の友セリヌンティウスと過ごした村には戻れない

まずは、太宰治版の物語に登場する名前にもとづいて、メロスが往復した先を調べてみる。※

メロスがシラクサ(Siracusa)の街に人質として残したのは、竹馬の友、つまり小さい頃から共に過ごした友人であるセリヌンティウスだ。
セリヌンティウスというのは、そのものズバリ「セリヌスの人」という名前だ。
ということは、メロスとセリヌンティウスが幼い頃に暮らしていた場所は、シチリア島の南西部、セリヌス（Selinous）だろう。

SiracusaとSelinunteの緯度経度を調べて、地図に描いてみるとこうなる。

※太宰治が書いた物語にもとづくが、今回は登場人物の名前のみに着目してみる。すなわち、メロスが向かった先が「シラクスから十里先」という記述は、ひとまず忘れておくことにする。

In [1]:
# Geocoderパッケージを使う
import geocoder  # conda install conda-forge::geocoder
import time

# 地名文字列
place1='Siracusa,Italy'
place2='Selinunte,Italy'

# 緯度経度を取得する
# 実際に取得する時
ua={"User-Agent": "sample for python"}
siracusa=geocoder.osm(place1, timeout=5.0,headers=ua).latlng
time.sleep(3)
selinunte=geocoder.osm(place2, timeout=5.0,headers=ua).latlng

#jupyter{book}ビルド用
#siracusa=[37.075474, 15.286586]
#selinunte=[37.583, 12.817]

# 確認表示
print(place1, siracusa)
print(place2, selinunte)

Siracusa,Italy [37.0646139, 15.2907196]
Selinunte,Italy [37.5884265, 12.82465071085111]


In [2]:
# foliumパッケージを使う
import folium

#lat1,lon1 = siracusa.latlng
#lat2,lon2 = selinunte.latlng
lat1,lon1 = siracusa
lat2,lon2 = selinunte

# 地図を作る
fmap = folium.Map(location=siracusa,zoom_start=6)
#fmap = folium.Map(location=siracusa.latlng,zoom_start=6)

# マーカープロット
folium.Marker(location=[lat1,lon1],
              icon=folium.Icon(color='black')
             ).add_to(fmap)
folium.Marker(location=[lat2,lon2],icon=folium.Icon(color='blue')
             ).add_to(fmap)
fmap # 表示する

SiracusaとSelinunteは、シチリア島の中でもほとんど東西の反対側に位置していて、かなり遠い。
実際、距離を計算してみると、直線距離でも226kmほどもある。

In [3]:
# geopyパッケージを使う
from geopy.distance import geodesic # conda install conda-forge::geopy

distance_km = geodesic(siracusa, selinunte).km
print(distance_km,'(km)')

226.1579735777327 (km)


このシラクス（Siracusa）からSelinunteまでを距離226km×2=452kmを、メロスは往復したことになる。

日暮れが遅い時期、たとえば夏至の頃のシチリア島での「日暮れ」を調べると、およそ18時30分くらいだ。
すると、たとえば当日の日暮れまでの往復だと、直線距離でも時速53kmという非現実的な速度でなければ、メロスは往復できない。
タイムリミットが、太宰が描いたような三日後の日暮れまでであれば、夜も昼も走り続けること前提の上で、時速5.6kmという現実的な速さになる。
しかし、三日三晩不眠不休で走り続けるというのは、実際問題無理だろう。

In [4]:
# PyEphem astronomy libraryを使う
import ephem # conda install anaconda::ephem
import datetime

siracusa_obs = ephem.Observer()
siracusa_obs.lon = str(siracusa[0])
siracusa_obs.lat = str(siracusa[1])
siracusa_obs.date = ephem.Date('2024/06/21 13:0:0.0')
sun = ephem.Sun()

print('日の出時間：', siracusa_obs.next_rising(sun))
print('日暮れ時間：', siracusa_obs.next_setting(sun))

import math

h=siracusa[0]/15.
m=(h-math.floor(h))*60
h=math.floor(h)

print('＋',h,'時間',int(m),'分')

print('当日日暮れまでだった場合',226*2/8.5,'km/h')
print('三日後の日暮れだった場合',226*2/(24*3+8.5),'km/h')

日の出時間： 2024/6/22 03:02:34
日暮れ時間： 2024/6/21 16:04:56
＋ 2 時間 28 分
当日日暮れまでだった場合 53.1764705882353 km/h
三日後の日暮れだった場合 5.614906832298137 km/h


## 文献を調べて計算をする検証の旅は続く

さまざまな文献を辿っていくと、メロス（多くの文献ではピュティアス）が友人ダモンを人質として往復したのは、200ｋｍ以上も離れたSelinunteではなくて、また太宰が書いた十里先の村でもなくて、Siracusa市内のようだ。※

「走れメロス」の検証の旅は、まだ始まったばかり。
過去の文献を調べ、さまざまな計算をしていけば、いつかきっと何か発見することができるかもしれない。

※そうであれば、当日の日暮れまでというタイムリミットでも自然かもしれない。